In [59]:
import os
from typing import List
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
ROOT_DIR : str = Path(os.getcwd())
DATA_DIR : str = ROOT_DIR/"data"

In [2]:
def create_folder(path:str)->None:
    if not os.path.exists(path): 
        os.makedirs(path)
def get_subdirectories(parent_directory:str)->List[str]:
    subdirectories = []
    for item in os.listdir(parent_directory):
        item_path = os.path.join(parent_directory, item)
        subdirectories.append(item_path)
    return subdirectories

In [3]:
def load_files()->pd.DataFrame:
    list_of_files : List[pd.DataFrame] = []
    list_of_dirs = get_subdirectories(DATA_DIR)
    for item in list_of_dirs:
        list_of_files.append(pd.read_parquet(item))
    return pd.concat(list_of_files)

In [4]:
df_sih: pd.DataFrame = load_files()

In [7]:
print("\nNomes das colunas como uma lista:")
print(list(df_sih.columns))


Nomes das colunas como uma lista:
['UF_ZI', 'ANO_CMPT', 'MES_CMPT', 'ESPEC', 'CGC_HOSP', 'N_AIH', 'IDENT', 'CEP', 'MUNIC_RES', 'NASC', 'SEXO', 'UTI_MES_IN', 'UTI_MES_AN', 'UTI_MES_AL', 'UTI_MES_TO', 'MARCA_UTI', 'UTI_INT_IN', 'UTI_INT_AN', 'UTI_INT_AL', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'PROC_SOLIC', 'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_SADT', 'VAL_RN', 'VAL_ACOMP', 'VAL_ORTP', 'VAL_SANGUE', 'VAL_SADTSR', 'VAL_TRANSP', 'VAL_OBSANG', 'VAL_PED1AC', 'VAL_TOT', 'VAL_UTI', 'US_TOT', 'DT_INTER', 'DT_SAIDA', 'DIAG_PRINC', 'DIAG_SECUN', 'COBRANCA', 'NATUREZA', 'NAT_JUR', 'GESTAO', 'RUBRICA', 'IND_VDRL', 'MUNIC_MOV', 'COD_IDADE', 'IDADE', 'DIAS_PERM', 'MORTE', 'NACIONAL', 'NUM_PROC', 'CAR_INT', 'TOT_PT_SP', 'CPF_AUT', 'HOMONIMO', 'NUM_FILHOS', 'INSTRU', 'CID_NOTIF', 'CONTRACEP1', 'CONTRACEP2', 'GESTRISCO', 'INSC_PN', 'SEQ_AIH5', 'CBOR', 'CNAER', 'VINCPREV', 'GESTOR_COD', 'GESTOR_TP', 'GESTOR_CPF', 'GESTOR_DT', 'CNES', 'CNPJ_MANT', 'INFEHOSP', 'CID_ASSO', 'CID_MORTE', 'COMPLEX', 'FINANC'

In [31]:
print("\nValores únicos em DURACAO:")
print(df_sih['QT_DIARIAS'].unique())


Valores únicos em DURACAO após a conversão:
[  1   2   4   3  20   0   9  11   6   8   7  10  27  12   5  14  13  18
  16  15  19  31  30  17  21  26  34  40  32  28  37  22  41  25  33  24
  29  63  43  23  35  68  39  45  36  44  56  62  48  46  51  66  38  60
  74  49  64  42  53  47  65  59  57  50  80  77  81 139  99  54  71 105
  52  98 134  86  70  69  73  82  61 107  83  55  87 109 108  95  67  76
  97  96  58 104 101 166  72 118  75  79 103  93  92  78  85 102 140 159
 115 112 117  90  89  88 125 114 150  91 120 129  94 123 192 181 121 110
 106  84 132 111 100 194 116 128 124 185 179]


In [44]:
print("Valores únicos em DIAG_PRINC:")
print(df_sih['DIAG_PRINC'].unique())

Valores únicos em DIAG_PRINC:
['L989' 'N941' 'D212' ... 'L904' 'N760' 'T175']


In [40]:
mean_hospitalization_by_disease = df_sih.groupby('DIAG_PRINC')['QT_DIARIAS'].mean()

In [45]:
print("\nTempo médio de internação por tipo de doença (DIAG_PRINC):")
print(mean_hospitalization_by_disease.sort_values(ascending=False))


Tempo médio de internação por tipo de doença (DIAG_PRINC):
DIAG_PRINC
K670    95.0
A238    66.0
Q439    64.2
D802    59.0
J383    59.0
        ... 
Q440     0.0
I401     0.0
I425     0.0
B387     0.0
P023     0.0
Name: QT_DIARIAS, Length: 5711, dtype: float64


In [55]:
mean_hospitalization_by_disease_and_city = df_sih.groupby(['DIAG_PRINC', 'UF_ZI'])['QT_DIARIAS'].mean().reset_index()


In [60]:
print("\nTempo médio de internação por doença e município:")
print(mean_hospitalization_by_disease_and_city)


Tempo médio de internação por doença e município:
      DIAG_PRINC   UF_ZI  QT_DIARIAS
0           A000  430000    3.724138
1           A000  430420    4.000000
2           A000  430460    6.000000
3           A000  430910    2.000000
4           A000  431490   10.000000
...          ...     ...         ...
36820       Z981  430460    1.000000
36821       Z982  430920    5.000000
36822       Z988  430000    4.600000
36823       Z988  430510    6.750000
36824       Z988  431490    9.500000

[36825 rows x 3 columns]
